## Постановка задачи

### Проблема:
Booking — американский поисковик-агрегатор, позволяющий бронировать места в ресторанах, отелях, арендовать авто и т.д. Одна из проблем компании — это нечестные отели, которые накручивают себе рейтинг. Одним из способов обнаружения таких отелей является построение модели, которая предсказывает рейтинг отеля. Если предсказания модели сильно отличаются от фактического результата, то, возможно, отель ведёт себя нечестно, и его стоит проверить.

### Задача:
Создать модель, которая должна предсказывать рейтинг отеля по данным сайта Booking на основе имеющихся в датасете данных. 

In [150]:
import pandas as pd
import category_encoders as ce
import re # регулярные выражения

## Информация о датасете

Первоначальная версия датасета содержит 17 полей со следующей информацией:

* hotel_address — адрес отеля;
* review_date — дата, когда рецензент разместил соответствующий отзыв;
* average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
* hotel_name — название отеля;
* reviewer_nationality — страна рецензента;
* negative_review — отрицательный отзыв, который рецензент дал отелю;
* review_total_negative_word_counts — общее количество слов в отрицательном отзыве;
* positive_review — положительный отзыв, который рецензент дал отелю;
* review_total_positive_word_counts — общее количество слов в положительном отзыве.
* reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;
* total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;
* total_number_of_reviews — общее количество действительных отзывов об отеле;
* tags — теги, которые рецензент дал отелю;
* days_since_review — количество дней между датой проверки и датой очистки;
* additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
* lat — географическая широта отеля;
* lng — географическая долгота отеля.

Размер таблицы: (386803;17)
Пропуски в данных: 'lat', 'lng'

In [151]:
hotels = pd.read_csv('hotels.csv')
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [152]:
hotels = pd.read_csv('hotels.csv')
hotels.head(5)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643
3,216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,34,9/22/2015,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotles...,11,8,10.0,"[' Leisure trip ', ' Solo traveler ', ' Standa...",681 day,48.888697,2.394540
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,3/5/2016,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Break...,20,10,9.6,"[' Business trip ', ' Couple ', ' Standard Dou...",516 day,52.385601,4.847060


Признаков типа object  а данной таблице всего 8 шт:
1. hotel_address ✓
2. review_date ✓
3. hotel_name ✓
4. reviewer_nationality проверить на корреляцию, не знаю чем может пригодиться
5. negative_review
6. positive_review
7. tags ✓
8. days_since_review ✓

### Подзадача:
Получить информацию из признаков типа object, которая может пригодиться для обучения модели.

## Обработка признаков типа object

### ✓1. Обработка признака review_date

In [153]:
# Преобразуем информацию о дате в формат datetime
hotels['review_date'] = pd.to_datetime(hotels['review_date'],yearfirst=True)

#Вынесем информацию о годе и месяце в отдельные признаки
hotels['review_year'] = hotels['review_date'].dt.year
hotels['review_month'] = hotels['review_date'].dt.month

#Удалим исходный признак
hotels = hotels.drop('review_date',axis=1)

### ✓2. Обработка признака tags

Признак 'tags' содержит информацию о тэгах пользователей, оставленных в каждом коментариии к отелю, инфромация в ячейках представлена в виде строки. Нам необходимо получить из строки список тэгов. 

In [154]:
# Функция принимает строку с тэгами и возвращает список тэгов
def get_tags_list(tags_str):

    tags_list = tags_str[1:-1] # удаляем скобки вначале и в конце строки
    tags_list = tags_list.replace("' ","") # удаляем кавычки вначале тэга
    tags_list = tags_list.replace(" '","") # удаляем кавычки вконце тэга
    tags_list = tags_list.split(', ') # разделяем строку на отдельные тэги
    return tags_list

hotels['tags'] = hotels['tags'].apply(get_tags_list)

Попробуем выяснить какие тэги пользователи оставляли чаще, а какие реже.

In [155]:
# Функция принимает на вход список тэгов
# и возвращает словарь тэгов, где значения - количество их повтров в ячейках таблицы

uniq_tags_dict = dict() # задаем словарь, в который будем добавлять тэги

def tags_count(tags_list):
    for elem in tags_list:
        if elem in uniq_tags_dict.keys():
            uniq_tags_dict[elem] = int(uniq_tags_dict.get(elem)) + 1 
        else:
            uniq_tags_dict[elem] = 1
    return(uniq_tags_dict)
    
hotels['tags'].apply(tags_count)

# Количество уникальных тэгов в таблице
print("Количество уникальных тэгов в таблице:",len(uniq_tags_dict))

# Отсоритируем словарь с тэгами по значению в порядке убывания (сохранение порядка элементов словаря
# не гарантируется в версиях Python старше 3.7) 

sorted_values = sorted(uniq_tags_dict.values(),reverse=True) # Создадим отсортированный список со значениями 
sorted_dict = {} # пустой словарь, куда будем добавлять отсортированные ключи со значениями

for i in sorted_values:
    for k in uniq_tags_dict.keys():
        if uniq_tags_dict[k] == i:
            sorted_dict[k] = uniq_tags_dict[k]
            break

print(sorted_dict)

Количество уникальных тэгов в таблице: 2368
{'Leisure trip': 313593, 'Submitted from a mobile device': 230778, 'Couple': 189212, 'Stayed 1 night': 145373, 'Stayed 2 nights': 100263, 'Solo traveler': 81235, 'Stayed 3 nights': 72000, 'Business trip': 61989, 'Group': 49088, 'Family with young children': 45836, 'Stayed 4 nights': 35748, 'Double Room': 26386, 'Standard Double Room': 24151, 'Superior Double Room': 23550, 'Family with older children': 19802, 'Deluxe Double Room': 18623, 'Double or Twin Room': 16824, 'Stayed 5 nights': 15611, 'Standard Double or Twin Room': 13062, 'Classic Double Room': 12716, 'Superior Double or Twin Room': 10238, '2 rooms': 9287, 'Stayed 6 nights': 7399, 'Standard Twin Room': 7325, 'Single Room': 7227, 'Twin Room': 6279, 'Stayed 7 nights': 5549, 'Executive Double Room': 4763, 'Classic Double or Twin Room': 4576, 'Superior Twin Room': 4540, 'Club Double Room': 4485, 'Deluxe Double or Twin Room': 4465, 'Queen Room': 4071, 'Deluxe King Room': 3993, 'Superior Qu

Итак, в таблице 2368 уникальных тэга, из них чаще всего используется тэг 'Leisure trip' - 313593 раз, 
реже всего тэг 'King Room with Sea View' - 1 раз. Тэги, которые редко используют пользователи, не несут для нас полезной информации поэтому можно оставить только те тэги, которые чаще встречаются в датасете.

In [156]:
# Составим список топ-20 тэгов
top_tags = list(sorted_dict.keys())[:20]

# По каждому тэгу создадим отдельный признак, который будет содержать информацию о наличии или отутсвии тэга 
for tag in top_tags:
    hotels[tag] = hotels.tags.apply(lambda x: 1 if tag in x else 0)
    
# Удаляем исходный столбец
hotels = hotels.drop('tags',axis=1)

### ✓3. Обработка признака days_since_review

Признак days_since_review содержит информацию о количестве прошедших дней с момента создания отзыва в формате строки, вынесем из этого признака число дней

In [157]:
# Функция принимает на вход строку и возвращает из нее число
def get_day_count(string):
    pattern = re.compile(r'\d+') #задаем паттерн для поиска числа в строке
    day_count = int(pattern.findall(string)[0]) #применяем паттерн и преобразуем строку в число
    return day_count

# Применим функцию к признаку days_since_review
hotels['days_since_review'] = hotels['days_since_review'].apply(get_day_count)

### 4. Обработка признака reviewer_nationality

Скорее всего этот признак не принесет статистической значимости, окончательный вывод можно будет сделать после тестирования на корреляцию

In [158]:
print("Количество уникальных значений",hotels['reviewer_nationality'].nunique())

Количество уникальных значений 225


In [159]:
# Топ 20 национальностей рецензентов
hotels['reviewer_nationality'].value_counts()[:20]

 United Kingdom               184033
 United States of America      26541
 Australia                     16239
 Ireland                       11126
 United Arab Emirates           7617
 Saudi Arabia                   6722
 Netherlands                    6608
 Switzerland                    6498
 Canada                         5984
 Germany                        5956
 France                         5543
 Israel                         4918
 Italy                          4562
 Belgium                        4529
 Turkey                         4105
 Kuwait                         3702
 Spain                          3529
 Romania                        3425
 Russia                         2953
 South Africa                   2888
Name: reviewer_nationality, dtype: int64

Для того чтобы опредедить нужен ли признак для предсказательной модели, необходимо произвести корреляционный анализ

### 5. Обработка признака hotel_address

В адрессе старана, где находится отель обозначена в конце адресса. Наша задача попробовать получить из этой строки только название станы, в которой находится отель.

Заметим, что национальность интервьюера записана в таком же формате, как название страны в адрессе отеля (т.е. без окончаний, только название страны), значит отсюда мы можем получить список из названий стран.

In [160]:
nat_list = list(hotels.reviewer_nationality.value_counts().index) # получаем названия национальностей
# Для корректного поиска удаляем пробел в конце каждоый строки
count_list = []
for nat in nat_list:
    nat = nat[:-1]
    count_list.append(nat)
# Получаем список названий стран
count_list

[' United Kingdom',
 ' United States of America',
 ' Australia',
 ' Ireland',
 ' United Arab Emirates',
 ' Saudi Arabia',
 ' Netherlands',
 ' Switzerland',
 ' Canada',
 ' Germany',
 ' France',
 ' Israel',
 ' Italy',
 ' Belgium',
 ' Turkey',
 ' Kuwait',
 ' Spain',
 ' Romania',
 ' Russia',
 ' South Africa',
 ' China',
 ' Sweden',
 ' India',
 ' Greece',
 ' New Zealand',
 ' Singapore',
 ' Hong Kong',
 ' Poland',
 ' Qatar',
 ' Austria',
 ' Egypt',
 ' Norway',
 ' Czech Republic',
 ' Lebanon',
 ' Hungary',
 ' Malaysia',
 ' Thailand',
 ' Brazil',
 ' Portugal',
 ' Finland',
 ' Denmark',
 ' Malta',
 ' Bahrain',
 ' Indonesia',
 ' Oman',
 ' Luxembourg',
 ' Croatia',
 ' Bulgaria',
 ' Cyprus',
 ' Japan',
 ' Serbia',
 ' Iran',
 ' Ukraine',
 ' Philippines',
 ' South Korea',
 ' Nigeria',
 ' Iceland',
 ' Slovakia',
 ' Taiwan',
 ' Pakistan',
 ' Jersey',
 ' Slovenia',
 ' Jordan',
 ' Lithuania',
 ' Estonia',
 ' Latvia',
 ' Mexico',
 ' Guernsey',
 '',
 ' Argentina',
 ' Isle of Man',
 ' Gibraltar',
 ' Chile'

In [161]:
# Функция принимает на вход строку и если название страны есть найдено в строке, 
# функция вернет только назвние страны, в которой находится отель.

def get_country(adress):
    for elem in count_list:
        if elem in adress:            
            return elem

# Получим новый признак с информацией о стране, в которой находится отель        
hotels['hotel_country'] = hotels['hotel_address'].apply(get_country) 
# Удалим признак с полным адрессом отеля
hotels = hotels.drop('hotel_address',axis=1)

In [162]:
print('Отели в датасете располагаются в',hotels['hotel_country'].nunique(),'странах')

Отели в датасете располагаются в 6 странах


Количество стран небольшое, значит данный признак можно будет в дальнейшем закодировать для обучения модели.

### ✓6. Обработка признака hotel_name

Названия некоторых отелей в таблице повторяются более 1500 раз, что говорит о том, что данные отели являются сетевыми.
На основе этой информации можно создать признак, который будет свидетельствовать о том, является отель частью большой сети отелей или нет.

In [163]:
# Создадим список топ 20 названий отелей по количеству отелей в сети
top_20_hotel_names = hotels.hotel_name.value_counts()[:20]
top_20_hotel_names = list(top_20_hotel_names.index)

# Признак chain_hotel содержит информацию о принадлежности 
# или непринадлежности отеля к большой сети
hotels['chain_hotel'] = hotels.hotel_name.apply(
    lambda x: 1 if x in top_20_hotel_names else 0
    )

# Удалаяем столбец hotel_name
hotels = hotels.drop('hotel_name',axis=1)

### 7. Обработка признака negative_review

In [164]:
hotels['negative_review'].value_counts()

No Negative                                                                                                                                                                                                                                                                                                                                                                    95907
 Nothing                                                                                                                                                                                                                                                                                                                                                                       10737
 Nothing                                                                                                                                                                                                                                                      

In [165]:
hotels[hotels['negative_review']=='Positive']['negative_review'].value_counts()

Series([], Name: negative_review, dtype: int64)

In [166]:
hotels[hotels['negative_review']==' Nothing']['review_total_negative_word_counts'].value_counts()

2    10737
Name: review_total_negative_word_counts, dtype: int64

In [167]:
hotels[hotels['review_total_negative_word_counts']<5]['negative_review'].value_counts()

No Negative          95907
 Nothing             10737
 Nothing              3154
 nothing              1660
 N A                   802
                     ...  
 Nothing especial        1
 Foam pillows            1
 checking in             1
 firm pillows            1
 Shower in bath          1
Name: negative_review, Length: 9970, dtype: int64

In [168]:
hotels[hotels['review_total_negative_word_counts']>12]['review_total_negative_word_counts'].value_counts()

13     7869
14     7474
15     6870
16     6681
17     6275
       ... 
400       1
402       1
398       1
408       1
392       1
Name: review_total_negative_word_counts, Length: 390, dtype: int64

### 8. Обработка признака positive_review

In [169]:
hotels[['review_total_negative_word_counts','positive_review']][:10]

,review_total_negative_word_counts,positive_review
0,3,Staff were amazing
1,3,location
2,6,No Positive
3,0,Friendly staff quiet comfortable room spotles...
4,4,The staff was very friendly and helpful Break...
5,2,Very nice hotel Price is excellent when compa...
6,6,Very central location 2 minutes walk from Glo...
7,40,The location is good for transport links espe...
8,30,staff very good location wonderful
9,47,No Positive


In [170]:
hotels['negative_review'] = hotels['negative_review'].apply(
    lambda x: x.replace('No Negative','Positive'))
hotels['negative_review'] = hotels['negative_review'].apply(
    lambda x: x.replace('N A','Positive'))
hotels['negative_review'] = hotels['negative_review'].apply(
    lambda x: x.replace('All good','Positive'))
hotels['negative_review'] = hotels['negative_review'].apply(
    lambda x: x.replace('No complaints','Positive'))
hotels['negative_review'] = hotels['negative_review'].apply(
    lambda x: x.replace('Nothing to dislike','Positive'))
hotels['negative_review'] = hotels['negative_review'].apply(
    lambda x: x.replace(' Nothing','Positive'))
hotels['negative_review'] = hotels['negative_review'].apply(
    lambda x: x.replace(' nothing','Positive'))
hotels['negative_review'] = hotels['negative_review'].apply(
    lambda x: x.replace('N a','Positive'))
hotels['negative_review'] = hotels['negative_review'].apply(
    lambda x: x.replace(' Positive','Positive'))
hotels['negative_review'] = hotels['negative_review'].apply(
    lambda x: x.replace('Positive really','Positive'))
hotels['negative_review'] = hotels['negative_review'].apply(
    lambda x: x.replace(' None','Positive'))

hotels['positive_review'] = hotels['positive_review'].apply(
    lambda x: x.replace('No Positive', 'Negative')) 

In [181]:

hotels['negative_review'] = hotels['negative_review'].apply(
    lambda x: x.replace('      ','Positive'))

In [188]:
hotels['negative_review'].value_counts()

Positive                                                                                                                                                                                                                                                                                                                                                                       111123
Positive                                                                                                                                                                                                                                                                                                                                                                         4082
                                                                                                                                                                                                                                                            